### Our Objective:🎯
- Use Keras Sequential/functioal API to define the model.
- Use [ImageDataset](https://www.tensorflow.org/datasets/catalog/caltech101) to train the model
  - Split the dataset into **train-test-valid**
- Post training, plot graphs 📉 of **loss & accuracy** for visualization.
- If possible try to perform hyperparameter tunning to further improve accuracy of the model.

[AlexNet Paper](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
from tensorflow.keras import layers, optimizers, metrics, losses

### Load & Split a Dataset
- Here we are using [oxford_flowers102](https://www.tensorflow.org/datasets/catalog/oxford_flowers102)

In [ ]:
(test_ds, train_ds, valid_ds), info_ds = tfds.load(
    'oxford_flowers102',
    split=['train', 'test', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete8ESOIJ/oxford_flowers102-train.tfrecord


...

  0%|          | 0/6149 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete8ESOIJ/oxford_flowers102-validation.tfrecord


  0%|          | 0/1020 [00:00<?, ? examples/s]

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


**NOTE: IN this dataset , num of examples in test > train. Hence use test_ds for train_ds** 😠

In [ ]:
# Please run this cell to get info.
info_ds.splits

{'test': <tfds.core.SplitInfo num_examples=6149>,
 'train': <tfds.core.SplitInfo num_examples=1020>,
 'validation': <tfds.core.SplitInfo num_examples=1020>}

In [ ]:
oxfd_num_classes = info_ds.features['label'].num_classes
oxfd_class_names = info_ds.features['label'].names

In [ ]:
print(oxfd_num_classes)

102


In [ ]:
print('len train_ds ',len(train_ds))
print('len test_ds ',len(test_ds))

len train_ds  6149
len test_ds  1020


### Create Train and Test Pipeline

In [ ]:
# PARAMETERS
BATCH_SIZE = 32

In [ ]:
def process_img(image, label):
  image = tf.image.per_image_standardization(image)
  image = tf.image.resize(image, (227,227))
  return image, label

train_ds = train_ds.map(process_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(800)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_ds = test_ds.map(process_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(800)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
VIZ_DS_FLAG = False

if VIZ_DS_FLAG == True:
  import matplotlib.pyplot as plt
  plt.figure(figsize=(10, 10))
  for images, labels in train_ds.take(1):
    for i in range(9):
      ax = plt.subplot(3, 3, i + 1)
      plt.imshow(images[i].numpy().astype("uint8"))

### Implement and Train the AlexNet model

In [ ]:
# conv--maxpool--conv--maxpool--conv--conv--conv--pool--dense--dense--dense
inputs = layers.Input(shape=(227,227,3))

x = layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

x = layers.Conv2D(filters=256, kernel_size=(5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

x = layers.Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096,activation='relu')(x)
x = layers.Dropout(0.5)(x)
# Output, since we are using oxford_flowers102, the number of classes is 102
# Hence we are using 102, instead of 1000 as in AlexNet Paper.
outputs = layers.Dense(oxfd_num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=optimizers.SGD(learning_rate=0.001),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024  

In [ ]:
history = model.fit(train_ds, epochs=50, validation_data=test_ds)

Epoch 1/50
193/193 [==============================] - 943s 5s/step - loss: 5.3482 - accuracy: 0.0568 - val_loss: 4.2714 - val_accuracy: 0.0686
Epoch 2/50
193/193 [==============================] - 938s 5s/step - loss: 4.0997 - accuracy: 0.1150 - val_loss: 3.8078 - val_accuracy: 0.1343
Epoch 3/50
193/193 [==============================] - 943s 5s/step - loss: 3.6808 - accuracy: 0.1667 - val_loss: 3.5662 - val_accuracy: 0.1667
Epoch 4/50
193/193 [==============================] - 888s 5s/step - loss: 3.4142 - accuracy: 0.2020 - val_loss: 3.3598 - val_accuracy: 0.1971
Epoch 5/50
193/193 [==============================] - 883s 5s/step - loss: 3.1670 - accuracy: 0.2496 - val_loss: 3.2330 - val_accuracy: 0.2235
Epoch 6/50
193/193 [==============================] - 883s 5s/step - loss: 2.9995 - accuracy: 0.2766 - val_loss: 3.0998 - val_accuracy: 0.2422
Epoch 7/50
193/193 [==============================] - 883s 5s/step - loss: 2.8239 - accuracy: 0.3017 - val_loss: 3.0216 - val_accuracy: 0.2490

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
model.save("alexnet_oxford_flower102")